# PS-3
## Name: Atharv Jagdish Patwardhan

## Research Topic: Data center placement and it's effects across the USA.
With the rise of AI across the world, the demand for high performance computing has increased, creating more demand for datacenters. There datacenters take up large plots of land and consume a lot of energy. In this notebook, we will look at factors that might affect the placement of these datacenters and also the impact they have on the environment.

In [ ]:
!pip install rasterio geopandas matplotlib requests folio branca

import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import requests, zipfile, io
from rasterio.mask import mask


In [ ]:
base_url = "https://raw.githubusercontent.com/atharvpatwardhan/Atharv-Patwardhan-GIS-Coursework/main/data/"

In [ ]:
import requests, zipfile, io, os

url = 'https://www.weather.gov/source/gis/Shapefiles/County/s_05mr24.zip'

zip_path = "data/usa_map.zip"
extract_dir = "data/usa_map"

os.makedirs("data", exist_ok=True)


if not os.path.exists(zip_path):
    r = requests.get(url)
    with open(zip_path, "wb") as f:
        f.write(r.content)
    print("Downloaded USA Map Data")

if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_dir)






In [ ]:
import os

for f in os.listdir('data/usa_map'):
    print(f)




In [ ]:
usa_data = gpd.read_file("data/usa_map/s_05mr24.shp")
print(usa_data.head())

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

usa_data = gpd.read_file("data/usa_map/s_05mr24.shp")

mainland_states = [
    "AL","AR","AZ","CA","CO","CT","DE","FL","GA","IA","ID","IL","IN",
    "KS","KY","LA","MA","MD","ME","MI","MN","MO","MS","MT","NC","ND",
    "NE","NH","NJ","NM","NV","NY","OH","OK","OR","PA","RI","SC","SD",
    "TN","TX","UT","VA","VT","WA","WI","WV","WY"
]

mainland_usa = usa_data[usa_data["STATE"].isin(mainland_states)]


## Electricity price and usage
As data centers are responsible for hosting everything from websites to cloud storage, they are extremely energy-intensive, accounting for about 2% of global electricity consumption. States with lower electricity costs—such as Washington (where hydroelectric power dominates) and Oregon—are hotspots for data centers, while states like Hawaii, where power costs are high due to reliance on imported fossil fuels, see fewer data center investments.

Did you know? The largest data center in the U.S. is the Meta Prinesville facility datacenter located in Prinesville, Oregon, spanning over 4.6 million square feet! It's a fascinating blend of tech, infrastructure, and geography that shapes how and where data centers thrive across the nation.

![My Image](https://github.com/atharvpatwardhan/Atharv-Patwardhan-GIS-Coursework/blob/main/images/image.jpeg?raw=true)


In [ ]:
import pandas as pd

url = "https://www.eia.gov/electricity/data/state/avgprice_annual.xlsx"

electricity_df = pd.read_excel(url, sheet_name="Price", skiprows=1)

print(electricity_df.head())


### Looking at the electricity cost per state in the USA

In [ ]:
import geopandas as gpd
import pandas as pd
import folium

usa_states = gpd.read_file("data/usa_map/s_05mr24.shp")
usa_states = usa_states[~usa_states["NAME"].isin(["Hawaii", "Alaska", "Puerto Rico"])]

generation = pd.read_excel(base_url + "generation_monthly.xlsx")

us_state_map = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California',
    'CO':'Colorado','CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia',
    'HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas',
    'KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts',
    'MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana',
    'NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico',
    'NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina',
    'SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont',
    'VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}
generation['State'] = generation['STATE'].map(us_state_map)

generation_state = (
    generation.groupby('State', as_index=False)['GENERATION (Megawatthours)']
    .sum()
    .rename(columns={'GENERATION (Megawatthours)': 'Generation_MWh'})
)

merged_geo = usa_states.merge(generation_state, left_on="NAME", right_on="State", how="left")

def load_datacenters(path, company):
    df = pd.read_csv(path)
    df = df[df["Country"].str.lower().str.strip() == "united states"]
    df = df[(df["Latitude"] != "-") & (df["Longitude"] != "-")]
    df["Latitude"] = df["Latitude"].astype(float)
    df["Longitude"] = df["Longitude"].astype(float)
    df["Company"] = company
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]),
        crs="EPSG:4326"
    )
    return gdf

amazon_gdf = load_datacenters(base_url + "amazon_data_centers.csv", "Amazon")
google_gdf = load_datacenters(base_url + "google_data_centers.csv", "Google")
meta_gdf = load_datacenters(base_url + "meta_data_centers.csv", "Meta")
microsoft_gdf = load_datacenters(base_url + "microsoft_data_centers.csv", "Microsoft")

datacenters = pd.concat([amazon_gdf, google_gdf, meta_gdf, microsoft_gdf])

m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="cartodb positron")

folium.Choropleth(
    geo_data=merged_geo,
    name="Electricity Generation (MWh)",
    data=merged_geo,
    columns=["NAME", "Generation_MWh"],
    key_on="feature.properties.NAME",
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.3,
    legend_name="Electricity Generation (MWh)"
).add_to(m)

company_colors = {
    "Amazon": "blue",
    "Google": "green",
    "Meta": "purple",
    "Microsoft": "orange"
}

for company, color in company_colors.items():
    company_df = datacenters[datacenters["Company"] == company]
    for _, row in company_df.iterrows():
        popup_html = f"""
        <b>{company} Data Center</b><br>
        Location: {row.get('City', 'Unknown')}<br>
        Lat: {row['Latitude']:.2f}, Lon: {row['Longitude']:.2f}
        """
        folium.CircleMarker(
            location=[row["Latitude"], row["Longitude"]],
            radius=6,
            color="black",
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=popup_html
        ).add_to(m)

legend_html = """
<div style="
     position: fixed;
     bottom: 40px; left: 40px; width: 250px; height: 140px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     box-shadow: 2px 2px 5px rgba(0,0,0,0.3); padding: 10px;">
<b>Legend</b><br>
<span style="background-color:#ffcc00;width:20px;height:20px;display:inline-block;margin-right:5px;"></span> Electricity Generation<br>
<span style="background-color:blue;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Amazon<br>
<span style="background-color:green;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Google<br>
<span style="background-color:purple;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Meta<br>
<span style="background-color:orange;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Microsoft
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

m.save("interactive_map.html")
m


In [ ]:
from google.colab import files

files.download('interactive_map.html')
files.download('unemployment_datacenters_map.html')

In [ ]:
url = "https://www.eia.gov/electricity/data/state/avgprice_annual.xlsx"

electricity_df = pd.read_excel(url, sheet_name="Price", skiprows=1)

electricity_df.head()
electricity_df.rename(columns={"State": "STATE"}, inplace=True)
electricity_df.head()

In [ ]:
import pandas as pd

dc_df = pd.read_csv(base_url + "datacenter_counts_by_state.csv")

us_abbrev = {
    "Alabama":"AL", "Alaska":"AK", "Arizona":"AZ", "Arkansas":"AR", "California":"CA", "Colorado":"CO",
    "Connecticut":"CT", "Delaware":"DE", "Florida":"FL", "Georgia":"GA", "Hawaii":"HI", "Idaho":"ID",
    "Illinois":"IL", "Indiana":"IN", "Iowa":"IA", "Kansas":"KS", "Kentucky":"KY", "Louisiana":"LA",
    "Maine":"ME", "Maryland":"MD", "Massachusetts":"MA", "Michigan":"MI", "Minnesota":"MN",
    "Mississippi":"MS", "Missouri":"MO", "Montana":"MT", "Nebraska":"NE", "Nevada":"NV",
    "New Hampshire":"NH", "New Jersey":"NJ", "New Mexico":"NM", "New York":"NY", "North Carolina":"NC",
    "North Dakota":"ND", "Ohio":"OH", "Oklahoma":"OK", "Oregon":"OR", "Pennsylvania":"PA",
    "Rhode Island":"RI", "South Carolina":"SC", "South Dakota":"SD", "Tennessee":"TN", "Texas":"TX",
    "Utah":"UT", "Vermont":"VT", "Virginia":"VA", "Washington":"WA", "West Virginia":"WV",
    "Wisconsin":"WI", "Wyoming":"WY"
}
dc_df["STATE"] = dc_df["State"].map(us_abbrev)

merged_electricity = pd.merge(dc_df, electricity_df, on="STATE", how="left")

merged_electricity.head()


In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear

usa_states = gpd.read_file("data/usa_map/s_05mr24.shp")
usa_states = usa_states[~usa_states["NAME"].isin(["Hawaii", "Alaska", "Puerto Rico"])]
centroids = usa_states.geometry.centroid
usa_states["centroid_lat"] = centroids.y
usa_states["centroid_lon"] = centroids.x

elec_price_state = (
    merged_electricity[merged_electricity["Industry Sector Category"] == "Total Electric Industry"]
    .groupby("State", as_index=False)["Commercial"]
    .mean()
    .rename(columns={"Commercial": "ElectricityPrice"})
)


dc_count_state = (
    merged_electricity.groupby("State", as_index=False)["DataCenters"]
    .max()
)

map_df = elec_price_state.merge(dc_count_state, on="State", how="left")
geo_map_df = usa_states.merge(map_df, left_on="NAME", right_on="State", how="left")


m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="CartoDB Positron")


colormap = linear.YlOrRd_09.scale(
    geo_map_df["ElectricityPrice"].min(),
    geo_map_df["ElectricityPrice"].max()
)
colormap.caption = "Electricity Price (cents per kWh)"

folium.Choropleth(
    geo_data=geo_map_df,
    name="Electricity Price (Commercial)",
    data=geo_map_df,
    columns=["NAME", "ElectricityPrice"],
    key_on="feature.properties.NAME",
    fill_color="YlOrRd",
    fill_opacity=0.85,
    line_opacity=0.5,
    legend_name="Electricity Price (cents per kWh)"
).add_to(m)

colormap.add_to(m)


max_dc = geo_map_df["DataCenters"].max()
for _, row in geo_map_df.iterrows():
    if pd.notnull(row["DataCenters"]):
        color = "red"
        radius = 5 + (row["DataCenters"] ** 0.5)
        folium.CircleMarker(
            location=[row["centroid_lat"], row["centroid_lon"]],
            radius=radius,
            color="red",
            fill=True,
            fill_color=color,
            fill_opacity=1,
            popup=f"{row['NAME']}<br>Datacenters: {int(row['DataCenters'])}<br>Electricity Price: {row['ElectricityPrice']:.2f}"
        ).add_to(m)


legend_html = """
<div style="
     position: fixed; bottom: 40px; left: 40px; width:260px;
     background:white; border:2px solid grey; z-index:9999;
     font-size:14px; padding:12px; border-radius:10px;">
<b>Legend</b><br>
<span style="background: linear-gradient(to right, yellow, red); width:50px; height:15px;
      display:inline-block; margin-right:8px;"></span> Datacenter Count (circle size & color)
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))


m.save("datacenter_price_map.html")
m


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D

datacenters_df = pd.read_csv(base_url + "datacenter_counts_by_state.csv")
electricity_df = pd.read_excel(base_url + "generation_monthly.xlsx")

us_state_map = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California',
    'CO':'Colorado','CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia',
    'HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas',
    'KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts',
    'MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana',
    'NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico',
    'NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina',
    'SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont',
    'VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

electricity_df['State'] = electricity_df['STATE'].map(us_state_map)

electricity_state = (
    electricity_df.groupby('State', as_index=False)['GENERATION (Megawatthours)']
    .sum()
    .rename(columns={'GENERATION (Megawatthours)': 'Generation_MWh'})
)

merged_df = pd.merge(datacenters_df, electricity_state, on='State', how='left')

usa_states = gpd.read_file("data/usa_map/s_05mr24.shp")
usa_states = usa_states[~usa_states["NAME"].isin(["Hawaii", "Alaska", "Puerto Rico"])]

merged_geo = usa_states.merge(merged_df, left_on="NAME", right_on="State", how="left")

fig, ax = plt.subplots(figsize=(14, 9))
ax.set_facecolor("#F9F9F9")

merged_geo.plot(
    column='Generation_MWh',
    cmap='YlGnBu',  # teal-blue map
    linewidth=0.8,
    ax=ax,
    edgecolor='white',
    legend=True,
    legend_kwds={'label': "Electricity Generation (MWh)", 'shrink': 0.6}
)

norm = Normalize(vmin=merged_geo['DataCenters'].min(), vmax=merged_geo['DataCenters'].max())
cmap = plt.cm.Reds

for _, row in merged_geo.dropna(subset=['DataCenters']).iterrows():
    plt.scatter(
        row.geometry.centroid.x,
        row.geometry.centroid.y,
        s=120,  # constant radius
        color=cmap(norm(row['DataCenters'])),
        alpha=0.75,
        edgecolor='black',
        linewidth=0.5
    )

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm._A = []  # required for colorbar
cbar = plt.colorbar(sm, ax=ax, shrink=0.6, pad=0.02)
cbar.set_label("Data Center Count", fontsize=10)

plt.title(
    "U.S. Electricity Generation vs Data Center Presence by State",
    fontsize=17,
    weight='bold',
    pad=20
)
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
cleaned = merged_electricity[
    merged_electricity['Industry Sector Category'] == 'Total Electric Industry'
]

cleaned = cleaned[['State', 'DataCenters', 'Total']].reset_index(drop=True)

cleaned = cleaned.rename(columns={'Total': 'avg_price'})


In [ ]:
import plotly.express as px
import pandas as pd

df = cleaned.copy()

# Compute quadrant thresholds
price_med = df['avg_price'].median()
dc_med = df['DataCenters'].median()

def classify(row):
    if row['avg_price'] >= price_med and row['DataCenters'] >= dc_med:
        return "High Price / High DC Count"
    elif row['avg_price'] < price_med and row['DataCenters'] >= dc_med:
        return "Low Price / High DC Count"
    elif row['avg_price'] >= price_med and row['DataCenters'] < dc_med:
        return "High Price / Low DC Count"
    else:
        return "Low Price / Low DC Count"

df['Quadrant'] = df.apply(classify, axis=1)

fig = px.scatter(
    df,
    x="avg_price",
    y="DataCenters",
    color="Quadrant",
    hover_name="State",
    hover_data={
        "avg_price": True,
        "DataCenters": True,
        "Quadrant": True
    },
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=700,
    size=[12]*len(df)
)

fig.add_vline(x=price_med, line_dash="dash", line_color="gray")
fig.add_hline(y=dc_med, line_dash="dash", line_color="gray")

fig.update_layout(
    title="Electricity Price vs Data Centers (Hover to See State)",
    xaxis_title="Electricity Price (cents per kWh)",
    yaxis_title="Data Centers",
    plot_bgcolor="rgba(245,245,245,0.3)",
    legend_title="Quadrant"
)

fig.show()


## Datacenters and Unemployment

### This study investigates the relationship between data center locations and unemployment rates across U.S. states, exploring whether the rise of digital infrastructure contributes to local job markets. While data centers themselves are not major employers—often requiring only a few dozen permanent staff once operational—they can have a significant indirect economic impact. Their construction phases create hundreds of temporary jobs, and their presence often attracts tech companies, cloud service providers, and infrastructure suppliers to the region.

In [ ]:
from io import BytesIO

ers_csv_url = "https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/48747/Unemployment2023.csv"

resp = requests.get(ers_csv_url)
resp.raise_for_status()
ers_df = pd.read_csv(BytesIO(resp.content))

print(ers_df.columns.tolist())

In [ ]:
ers_df.head()

In [ ]:
ers_pivot = ers_df.pivot_table(
    index="State",
    columns="Attribute",
    values="Value",
    aggfunc="first"
).reset_index()

ers_pivot.columns.name = None
ers_pivot = ers_pivot.rename(columns={"State": "STATE"})
ers_pivot.head()

In [ ]:
import geopandas as gpd
import pandas as pd
import folium

usa_states = gpd.read_file("data/usa_map/s_05mr24.shp")
usa_states = usa_states[~usa_states["NAME"].isin(["Hawaii", "Alaska", "Puerto Rico"])]

us_state_map = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California',
    'CO':'Colorado','CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia',
    'HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas',
    'KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts',
    'MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana',
    'NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico',
    'NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina',
    'SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont',
    'VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

ers_pivot["State"] = ers_pivot["STATE"].map(us_state_map)
merged_geo = usa_states.merge(ers_pivot, left_on="NAME", right_on="State", how="left")

def load_datacenters(path, company):
    df = pd.read_csv(path)
    df = df[df["Country"].str.lower().str.strip() == "united states"]
    df = df[(df["Latitude"] != "-") & (df["Longitude"] != "-")]
    df["Latitude"] = df["Latitude"].astype(float)
    df["Longitude"] = df["Longitude"].astype(float)
    df["Company"] = company
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]),
        crs="EPSG:4326"
    )
    return gdf

amazon_gdf = load_datacenters(base_url + "amazon_data_centers.csv", "Amazon")
google_gdf = load_datacenters(base_url + "google_data_centers.csv", "Google")
meta_gdf = load_datacenters(base_url + "meta_data_centers.csv", "Meta")
microsoft_gdf = load_datacenters(base_url + "microsoft_data_centers.csv", "Microsoft")

datacenters = pd.concat([amazon_gdf, google_gdf, meta_gdf, microsoft_gdf])

m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="cartodb positron")

folium.Choropleth(
    geo_data=merged_geo,
    name="Unemployment Rate (2023)",
    data=merged_geo,
    columns=["NAME", "Unemployment_rate_2023"],
    key_on="feature.properties.NAME",
    fill_color="PuBuGn",
    fill_opacity=0.8,
    line_opacity=0.3,
    legend_name="Unemployment Rate (2023)",
).add_to(m)

company_colors = {
    "Amazon": "#1f77b4",
    "Google": "#2ca02c",
    "Meta": "#9467bd",
    "Microsoft": "#ff7f0e"
}

for company, color in company_colors.items():
    company_df = datacenters[datacenters["Company"] == company]
    for _, row in company_df.iterrows():
        popup_html = f"""
        <b>{company} Data Center</b><br>
        Location: {row.get('City', 'Unknown')}<br>
        Lat: {row['Latitude']:.2f}, Lon: {row['Longitude']:.2f}
        """
        folium.CircleMarker(
            location=[row["Latitude"], row["Longitude"]],
            radius=6,
            color="black",
            fill=True,
            fill_color=color,
            fill_opacity=0.85,
            popup=popup_html
        ).add_to(m)

legend_html = """
<div style="
     position: fixed;
     bottom: 40px; left: 40px; width: 260px; height: 160px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     box-shadow: 2px 2px 5px rgba(0,0,0,0.3); padding: 10px;">
<b>Legend</b><br>
<span style="background-color:#6baed6;width:20px;height:20px;display:inline-block;margin-right:5px;"></span> Unemployment Rate<br>
<span style="background-color:#1f77b4;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Amazon<br>
<span style="background-color:#2ca02c;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Google<br>
<span style="background-color:#9467bd;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Meta<br>
<span style="background-color:#ff7f0e;width:15px;height:15px;border-radius:50%;display:inline-block;margin-right:5px;"></span> Microsoft
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

m.save("unemployment_datacenters_map.html")
m


In [ ]:
dc_df.head()

In [ ]:
ers_pivot.head()

In [ ]:
df_merged = pd.merge(dc_df, ers_pivot, on="STATE", how="left")
df_merged.head()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.choropleth(
    df_merged,
    locations="STATE",
    locationmode="USA-states",
    color="Unemployment_rate_2023",
    color_continuous_scale="PuBuGn",
    scope="usa",
    title="U.S. Unemployment Rate vs Data Center Presence by State",
    labels={"Unemployment_rate_2023": "Unemployment Rate (%)"}
)

fig.add_scattergeo(
    locations=df_merged["STATE"],
    locationmode="USA-states",
    text=df_merged["STATE"],
    marker=dict(
        size=df_merged["DataCenters"] / df_merged["DataCenters"].max() * 35 + 6,
        color="cyan",
        opacity=0.85,
        line=dict(width=0.8, color="white")
    ),
    name="Data Centers"
)

fig.update_layout(
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        showland=True,
        landcolor="rgb(245, 245, 245)",
        subunitcolor="white",
        showlakes=True,
        lakecolor="rgb(230, 230, 230)"
    ),
    title_x=0.5,
    title_font=dict(size=20, family="Arial Black"),
    margin=dict(r=0, t=60, l=0, b=0),
    coloraxis_colorbar=dict(
        title="Unemployment Rate (%)",
        tickfont=dict(size=12),
        titlefont=dict(size=14)
    )
)

fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

df = df_merged.copy()

df = df[['State_x', 'DataCenters', 'Unemployment_rate_2023']].dropna()

df.rename(columns={
    'State_x': 'State',
    'Unemployment_rate_2023': 'Unemployment'
}, inplace=True)

unemp_med = df['Unemployment'].median()
dc_med = df['DataCenters'].median()

def classify(row):
    if row['Unemployment'] >= unemp_med and row['DataCenters'] >= dc_med:
        return "High Unemployment / High DC Count"
    elif row['Unemployment'] < unemp_med and row['DataCenters'] >= dc_med:
        return "Low Unemployment / High DC Count"
    elif row['Unemployment'] >= unemp_med and row['DataCenters'] < dc_med:
        return "High Unemployment / Low DC Count"
    else:
        return "Low Unemployment / Low DC Count"

df['Quadrant'] = df.apply(classify, axis=1)

fig = px.scatter(
    df,
    x="Unemployment",
    y="DataCenters",
    color="Quadrant",
    hover_name="State",
    hover_data={
        "Unemployment": True,
        "DataCenters": True,
        "Quadrant": True
    },
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=700,
    size=[12]*len(df)
)

fig.add_vline(x=unemp_med, line_dash="dash", line_color="gray")
fig.add_hline(y=dc_med, line_dash="dash", line_color="gray")

fig.update_layout(
    title="Unemployment Rate (2023) vs Data Centers — Quadrant Classifier",
    xaxis_title="Unemployment Rate (2023, %)",
    yaxis_title="Data Center Count",
    plot_bgcolor="rgba(245,245,245,0.3)",
    legend_title="Quadrant"
)

fig.show()


In [ ]:
import pandas as pd


ers_pivot["Unemployment_rate_2023"] = pd.to_numeric(ers_pivot["Unemployment_rate_2023"], errors="coerce")

unemp_latest = ers_pivot[["STATE", "Unemployment_rate_2023"]].rename(columns={"STATE": "State_Abbr"})

merged = pd.merge(dc_df, unemp_latest, left_on="STATE", right_on="State_Abbr", how="left")

merged = merged.drop(columns=["State_Abbr"])
merged.head()


import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

plot = sns.jointplot(
    x=merged["DataCenters"],
    y=merged["Unemployment_rate_2023"],
    kind="reg",
    color="#2E86AB",
    height=6
)

plot.set_axis_labels(
    xlabel="Number of Data Centers",
    ylabel="Unemployment Rate (2023, %)"
)
plot.fig.suptitle(
    "Relationship Between Number of Data Centers and Unemployment Rate (2023)",
    fontsize=13
)
plt.tight_layout()
plt.show()


### We observe that there is no extreme unemployment in the states where data centers exist. Though there is no obvious correlation between unemployment and number of datacenters, the more datacenters in a state, the less the unemployment is the overall trend, California being the exception.

### Most employment is generated during the construction of datacenters. Once the datacenter is built, not a lot of people are needed to operate them.

In [ ]:
import pandas as pd

# Literacy dataset
literacy = pd.read_csv(base_url + 'literacy_rates.csv')
literacy

In [ ]:
unemployment = pd.read_csv("https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/48747/Unemployment2023.csv")
unemployment.rename(columns={"State": "State", "Unemployment": "UnemploymentRate"}, inplace=True)
unemployment_pivot = unemployment.pivot_table(
    index="State",
    columns="Attribute",
    values="Value",
    aggfunc="first"
).reset_index()

us_state_map = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California',
    'CO':'Colorado','CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia',
    'HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas',
    'KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts',
    'MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana',
    'NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico',
    'NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina',
    'SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont',
    'VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

unemployment_pivot["State"] = unemployment_pivot["State"].map(us_state_map)

unemployment_pivot.columns.name = None
unemployment_pivot.head()

In [ ]:
import pandas as pd

unemployment_literacy_merged = literacy.merge(unemployment_pivot, on="State", how="inner")
unemployment_literacy_merged.head()

In [ ]:
import folium

geojson_url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"

m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="CartoDB positron")

folium.Choropleth(
    geo_data=geojson_url,
    name="Literacy Rate",
    data=unemployment_literacy_merged,
    columns=["State", "LiteracyRate"],
    key_on="feature.properties.name",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Literacy Rate Score"
).add_to(m)

for _, row in unemployment_literacy_merged.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]] if "Latitude" in unemployment_literacy_merged.columns else None,
        popup=f"<b>{row['State']}</b><br>Literacy: {row['LiteracyRate']}<br>Unemployment: {row['Unemployment_rate_2023']}%"
    )

m


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(unemployment_literacy_merged["LiteracyRate"], unemployment_literacy_merged["Unemployment_rate_2023"], s=80)

for _, row in unemployment_literacy_merged.iterrows():
    plt.text(row["LiteracyRate"] + 0.1, row["Unemployment_rate_2023"] + 0.02, row["State"])

plt.xlabel("Literacy Rate (Score)")
plt.ylabel("Unemployment Rate (%)")
plt.title("Literacy Rate vs Unemployment Rate (State-level)")
plt.grid(True)
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd

df = unemployment_literacy_merged.copy()

# Select relevant columns
df = df[['State', 'LiteracyRate', 'Unemployment_rate_2023']].dropna()

df.rename(columns={
    'Unemployment_rate_2023': 'Unemployment'
}, inplace=True)

# Compute medians for quadrant classification
lit_med = df['LiteracyRate'].median()
unemp_med = df['Unemployment'].median()

# Quadrant classification
def classify(row):
    if row['LiteracyRate'] >= lit_med and row['Unemployment'] >= unemp_med:
        return "High Literacy / High Unemployment"
    elif row['LiteracyRate'] >= lit_med and row['Unemployment'] < unemp_med:
        return "High Literacy / Low Unemployment"
    elif row['LiteracyRate'] < lit_med and row['Unemployment'] >= unemp_med:
        return "Low Literacy / High Unemployment"
    else:
        return "Low Literacy / Low Unemployment"

df['Quadrant'] = df.apply(classify, axis=1)

# Build scatter plot
fig = px.scatter(
    df,
    x="LiteracyRate",
    y="Unemployment",
    color="Quadrant",
    hover_name="State",
    hover_data={
        "LiteracyRate": True,
        "Unemployment": True,
        "Quadrant": True
    },
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=700,
    size=[12] * len(df)
)

# Add median lines
fig.add_vline(x=lit_med, line_dash="dash", line_color="gray")
fig.add_hline(y=unemp_med, line_dash="dash", line_color="gray")

fig.update_layout(
    title="Literacy Rate vs Unemployment Rate (2023) — Quadrant Classifier",
    xaxis_title="Literacy Rate",
    yaxis_title="Unemployment Rate (%)",
    plot_bgcolor="rgba(245,245,245,0.3)",
    legend_title="Quadrant"
)

fig.show()


In [ ]:
df_lit = unemployment_literacy_merged.copy()
df_dc = df_merged[['State_x', 'DataCenters']].copy().rename(columns={'State_x': 'State'})

# merge on state name
df3 = df_lit.merge(df_dc, on='State', how='left')

# keep only needed columns
df3 = df3[['State', 'LiteracyRate', 'Unemployment_rate_2023', 'DataCenters']].dropna()

df3.rename(columns={'Unemployment_rate_2023': 'Unemployment'}, inplace=True)


In [ ]:
import plotly.express as px

fig = px.scatter_3d(
    df3,
    x="LiteracyRate",
    y="Unemployment",
    z="DataCenters",
    color="DataCenters",
    color_continuous_scale="Viridis",
    hover_name="State",
    hover_data={
        "LiteracyRate": True,
        "Unemployment": True,
        "DataCenters": True
    },
    height=800,
    size=[8] * len(df3),
)

fig.update_layout(
    title="3D Scatterplot: Literacy Rate vs Unemployment vs Data Center Count",
    scene=dict(
        xaxis_title="Literacy Rate",
        yaxis_title="Unemployment Rate (%)",
        zaxis_title="Data Center Count"
    )
)

fig.show()


In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear


usa_states = gpd.read_file("data/usa_map/s_05mr24.shp")
usa_states = usa_states[~usa_states["NAME"].isin(["Hawaii", "Alaska", "Puerto Rico"])]

centroids = usa_states.geometry.centroid
usa_states["centroid_lat"] = centroids.y
usa_states["centroid_lon"] = centroids.x


map_df = unemployment_literacy_merged[["State", "LiteracyRate", "Unemployment_rate_2023"]]

geo_map_df = usa_states.merge(map_df, left_on="NAME", right_on="State", how="left")


m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="CartoDB Positron")

colormap_lit = linear.BuPu_09.scale(
    geo_map_df["LiteracyRate"].min(),
    geo_map_df["LiteracyRate"].max()
)
colormap_lit.caption = "Literacy Rate"
folium.Choropleth(
    geo_data=geo_map_df,
    name="Literacy Rate",
    data=geo_map_df,
    columns=["NAME", "LiteracyRate"],
    key_on="feature.properties.NAME",
    fill_color="BuPu",
    fill_opacity=0.8,
    line_opacity=0.5,
    legend_name="Literacy Rate"
).add_to(m)

colormap_lit.add_to(m)


max_unemp = geo_map_df["Unemployment_rate_2023"].max()

for _, row in geo_map_df.iterrows():
    if pd.notnull(row["Unemployment_rate_2023"]):
        radius = 3 + (row["Unemployment_rate_2023"] / max_unemp) * 20

        intensity = row["Unemployment_rate_2023"] / max_unemp
        color = linear.OrRd_09.scale(
            geo_map_df["Unemployment_rate_2023"].min(),
            geo_map_df["Unemployment_rate_2023"].max()
        )(row["Unemployment_rate_2023"])

        folium.CircleMarker(
            location=[row["centroid_lat"], row["centroid_lon"]],
            radius=radius,
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"{row['NAME']}<br>Unemployment Rate: {row['Unemployment_rate_2023']:.1f}%"
        ).add_to(m)


legend_html = """
<div style="
     position: fixed; bottom: 40px; left: 40px; width:250px;
     background:white; border:2px solid grey; z-index:9999;
     font-size:14px; padding:10px; border-radius:12px;">
<b>Marker Legend</b><br>
<span style="background: linear-gradient(to right, orange, red); width:40px; height:15px;
      display:inline-block; margin-right:5px;"></span> Unemployment Rate (circle size & color)
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))


m.save("literacy_unemployment_map.html")
m


## Data centers and water proximity

Data centers are extremely water-intensive facilities, and their location decisions often align closely with access to reliable water sources. Cooling servers generates enormous heat, and many data centers rely on water-based cooling systems that can consume millions of gallons per day. As a result, regions with major rivers, aquifers, or stable freshwater infrastructure tend to attract more data-center development. However, this creates tension in areas already facing water scarcity or ecosystem stress. By comparing data-center locations with water sediment concentrations, we can explore whether environmental water quality—or proximity to specific water bodies—plays a role in where these facilities are being built. This analysis helps highlight potential sustainability risks and regional trade-offs tied to the rapid growth of digital infrastructure.

Fun Fact:
A single large data center can use between 3 million and 5 million gallons of water per day—roughly the same daily water consumption as a city of 30,000–50,000 people.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

usa_data = gpd.read_file("data/usa_map/s_05mr24.shp")

mainland_states = [
    "AL","AR","AZ","CA","CO","CT","DE","FL","GA","IA","ID","IL","IN",
    "KS","KY","LA","MA","MD","ME","MI","MN","MO","MS","MT","NC","ND",
    "NE","NH","NJ","NM","NV","NY","OH","OK","OR","PA","RI","SC","SD",
    "TN","TX","UT","VA","VT","WA","WI","WV","WY"
]
mainland_usa = usa_data[usa_data["STATE"].isin(mainland_states)]

dc_df = pd.read_csv(base_url + "datacenter_counts_by_state.csv")

us_abbrev = {
    "Alabama":"AL", "Alaska":"AK", "Arizona":"AZ", "Arkansas":"AR", "California":"CA", "Colorado":"CO",
    "Connecticut":"CT", "Delaware":"DE", "Florida":"FL", "Georgia":"GA", "Hawaii":"HI", "Idaho":"ID",
    "Illinois":"IL", "Indiana":"IN", "Iowa":"IA", "Kansas":"KS", "Kentucky":"KY", "Louisiana":"LA",
    "Maine":"ME", "Maryland":"MD", "Massachusetts":"MA", "Michigan":"MI", "Minnesota":"MN",
    "Mississippi":"MS", "Missouri":"MO", "Montana":"MT", "Nebraska":"NE", "Nevada":"NV",
    "New Hampshire":"NH", "New Jersey":"NJ", "New Mexico":"NM", "New York":"NY", "North Carolina":"NC",
    "North Dakota":"ND", "Ohio":"OH", "Oklahoma":"OK", "Oregon":"OR", "Pennsylvania":"PA",
    "Rhode Island":"RI", "South Carolina":"SC", "South Dakota":"SD", "Tennessee":"TN", "Texas":"TX",
    "Utah":"UT", "Vermont":"VT", "Virginia":"VA", "Washington":"WA", "West Virginia":"WV",
    "Wisconsin":"WI", "Wyoming":"WY"
}
dc_df["STATE"] = dc_df["State"].map(us_abbrev)

merged = mainland_usa.merge(dc_df, on="STATE", how="left")

fig, ax = plt.subplots(figsize=(14, 10))
merged.plot(
    column="DataCenters",
    cmap="Blues",
    linewidth=0.8,
    edgecolor="black",
    legend=True,
    vmin=0,
    vmax=400,
    legend_kwds={
        "label": "Number of Data Centers",
        "orientation": "horizontal",
        "shrink": 0.6
    },
    ax=ax
)

ax.set_title("Data Center Distribution Across the U.S.", fontsize=18, pad=20)
ax.axis("off")

plt.show()


In [ ]:
dc_df.head()

In [ ]:
import geopandas as gpd
import pandas as pd

sediments = pd.read_csv(base_url + "SedimentDatabase_Locations.csv")

usa_sed = sediments[sediments["Country"].str.contains("USA", na=False)].copy()
usa_sed = usa_sed.dropna(subset=["Lat_deg", "Lon_deg"])

gdf_sed = gpd.GeoDataFrame(
    usa_sed,
    geometry=gpd.points_from_xy(usa_sed["Lon_deg"], usa_sed["Lat_deg"]),
    crs="EPSG:4326"
)

gdf_sed_mainland = gpd.sjoin(gdf_sed, mainland_usa[["STATE", "geometry"]], how="inner", predicate="within")


fig, ax = plt.subplots(figsize=(14, 10))

merged.plot(
    column="DataCenters",
    cmap="Blues",
    linewidth=0.8,
    edgecolor="black",
    legend=True,
    legend_kwds={
        "label": "Number of Data Centers",
        "orientation": "horizontal",
        "shrink": 0.6
    },
    ax=ax
)

gdf_sed_mainland.plot(
    ax=ax,
    color="red",
    markersize=20,
    alpha=0.6,
    label="Sediment / Water Sites"
)

ax.set_title("Mainland U.S. Data Centers vs. Sediment (Water-Proximal) Locations", fontsize=18, pad=20)
ax.axis("off")
plt.legend()
plt.show()


In [ ]:
import folium
from folium import Choropleth, LayerControl, FeatureGroup, CircleMarker
import geopandas as gpd
import pandas as pd

sediments = pd.read_csv(base_url + "SedimentDatabase_Locations.csv")
usa_sed = sediments[sediments["Country"].str.contains("USA", na=False)].copy()
usa_sed = usa_sed.dropna(subset=["Lat_deg", "Lon_deg"])

gdf_sed = gpd.GeoDataFrame(
    usa_sed,
    geometry=gpd.points_from_xy(usa_sed["Lon_deg"], usa_sed["Lat_deg"]),
    crs="EPSG:4326"
)

gdf_sed_mainland = gpd.sjoin(
    gdf_sed,
    mainland_usa[["STATE", "geometry"]],
    how="inner",
    predicate="within"
)

m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="CartoDB positron")

Choropleth(
    geo_data=merged.to_json(),
    name="Data Centers by State",
    data=merged,
    columns=["STATE", "DataCenters"],
    key_on="feature.properties.STATE",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Number of Data Centers"
).add_to(m)

sediment_layer = FeatureGroup(name="Sediment / Water Sites")

for _, row in gdf_sed_mainland.iterrows():
    CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.5,
        popup=f"Site: {row.get('SiteName', 'N/A')}<br>State: {row['STATE']}"
    ).add_to(sediment_layer)

sediment_layer.add_to(m)

LayerControl(collapsed=False).add_to(m)

title_html = """
<h3 align="center" style="font-size:20px">
Mainland U.S. Data Centers vs. Sediment (Water-Proximal) Locations
</h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

m


In [ ]:
electricity_df = pd.read_excel(base_url + "generation_monthly.xlsx")
print(electricity_df.columns)


In [ ]:
electricity_df["STATE"].unique()[:20]


In [ ]:
import geopandas as gpd
import pandas as pd

usa_states = gpd.read_file("data/usa_map/s_05mr24.shp")
usa_states = usa_states[~usa_states["NAME"].isin(["Hawaii", "Alaska", "Puerto Rico"])]

generation = pd.read_excel(base_url + "generation_monthly.xlsx")
datacenters = pd.read_csv(base_url + "datacenter_counts_by_state.csv")


us_state_map = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California',
    'CO':'Colorado','CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia',
    'HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas',
    'KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts',
    'MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana',
    'NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico',
    'NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina',
    'SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont',
    'VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

generation['State'] = generation['STATE'].map(us_state_map)

generation_state = (
    generation.groupby('State', as_index=False)['GENERATION (Megawatthours)']
    .sum()
    .rename(columns={'GENERATION (Megawatthours)': 'Generation_MWh'})
)

merged_df = pd.merge(datacenters, generation_state, on='State', how='left')


merged_geo = usa_states.merge(merged_df, left_on="NAME", right_on="State", how="left")



In [ ]:
dc = pd.read_csv(base_url + "datacenter_counts_by_state.csv")

dc.head()

## Data centers and land prices

Data centers require huge footprints—not just for the servers, but also for cooling infrastructure, power distribution networks, and physical security buffers. Because of this, states with lower land prices often become hotspots for large-scale data center construction. Cheaper land directly reduces upfront capital expenditure and enables companies to build sprawling single-story facilities with room for future expansion. In contrast, states with expensive land tend to see fewer hyperscale facilities and more compact, high-rise data centers. This relationship between land cost and facility density is a major factor in where cloud providers choose to expand.

Fun Fact:
A typical hyperscale data center campus can require 100–250 acres of land to support server halls, cooling plants, substations, and redundancy systems.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium

land = pd.read_csv(base_url + "real_estate_values.csv")
dc = pd.read_csv(base_url + "datacenter_counts_by_state.csv")

land["State"] = land["State"].str.strip()
dc["State"] = dc["State"].str.strip()

merged = land.merge(dc, on="State")


url = "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json"
states_gdf = gpd.read_file(url)

states_gdf["name"] = states_gdf["name"].str.strip()

geo_merged = states_gdf.merge(merged, left_on="name", right_on="State")

m = folium.Map(location=[37.8, -96], zoom_start=5)

folium.Choropleth(
    geo_data=geo_merged,
    name="Land Price per Acre",
    data=geo_merged,
    columns=["State", "Price_per_Acre"],
    key_on="feature.properties.name",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Land Price Per Acre (USD)"
).add_to(m)


for _, row in geo_merged.iterrows():
    centroid = row["geometry"].centroid
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=max(row["DataCenters"]*0.08, 2),   # scale bubble size
        popup=f"{row['State']}: {row['DataCenters']} Data Centers",
        fill=True,
        fill_opacity=0.8
    ).add_to(m)
m.save("data_centers_and _land_prices.html")
m


Virginia, Texas, and California have the most data centers in the U.S., largely because of Virginia's proximity to government agencies and dense fiber optics in the "Data Center Alley" near Washington D.C., Texas's central location and lower costs, and California's concentration of tech companies in Silicon Valley. Other factors include the availability of land, power, water, and favorable tax incentives in many of these locations.

1.  **Virginia**
    *   **Proximity to government and dense fiber:** Northern Virginia, specifically "Data Center Alley" in Loudoun County, is the top hub due to its close ties with federal government agencies and a dense network of fiber optic cables.
    *   **Connectivity:** The region has four subsea fiber cables terminating there, providing direct connectivity to global markets.
    *   **Other advantages:** There is a skilled workforce, available land, and robust power and water supplies to support operations.

2.  **Texas**
    *   **Strategic location:** Primarily concentrated in the Dallas/Forth Worth area, Texas benefits from being a central hub for the nation.
    *   **Cost-effectiveness:** The state offers a lower cost of land and operations compared to coastal hubs.
    *   **Economic development:** Many states, including Texas, have created economic development zones to attract data centers.

3.  **California**
    *   **Silicon Valley hub:** California's high concentration of data centers is centered in Silicon Valley, a major technology hub.
    *   **Existing infrastructure:** The state has a high demand for data services from tech companies.
    *   **Drawbacks:** Despite the high concentration, California faces challenges like higher costs and regulatory hurdles.


In [ ]:
import plotly.express as px
import pandas as pd

df = merged.copy()

price_med = df['Price_per_Acre'].median()
dc_med = df['DataCenters'].median()

def classify(row):
    if row['Price_per_Acre'] >= price_med and row['DataCenters'] >= dc_med:
        return "High Price / High Count"
    elif row['Price_per_Acre'] >= price_med and row['DataCenters'] < dc_med:
        return "High Price / Low Count"
    elif row['Price_per_Acre'] < price_med and row['DataCenters'] >= dc_med:
        return "Low Price / High Count"
    else:
        return "Low Price / Low Count"

df['Quadrant'] = df.apply(classify, axis=1)

fig = px.scatter(
    df,
    x="Price_per_Acre",
    y="DataCenters",
    color="Quadrant",
    hover_name="State",
    hover_data={
        "Price_per_Acre": True,
        "DataCenters": True,
        "Quadrant": True
    },
    color_discrete_sequence=px.colors.qualitative.Pastel,
    height=700,
    size=[12] * len(df)
)

fig.add_vline(x=price_med, line_dash="dash", line_color="gray")
fig.add_hline(y=dc_med, line_dash="dash", line_color="gray")

fig.update_layout(
    title="Land Price per Acre vs Datacenter Count — Quadrant Classifier",
    xaxis_title="Land Price per Acre ($)",
    yaxis_title="Number of Datacenters",
    plot_bgcolor="rgba(245,245,245,0.3)",
    legend_title="Quadrant"
)

fig.write_html("land_price_vs_datacenters.html")

fig.show()


### Count-wise internet connection meeting

In [ ]:
import requests
import json

geojson_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
county_geojson = requests.get(geojson_url).json()

In [ ]:
df = pd.read_csv(base_url + "county_connections_200906_202406.csv", encoding="cp1252")
df["countycode"] = df["countycode"].astype(str).str.zfill(5)

latest = df[df["year"] == 2024]

In [ ]:
latest.head()

In [ ]:
latest["all"] = pd.to_numeric(latest["all"], errors="coerce")
latest.loc[latest["all"] < 0, "all"] = None

In [ ]:
import numpy as np

bins = list(latest["all"].quantile([0, 0.25, 0.5, 0.75, 1]))


In [ ]:
m = folium.Map(location=[37.8, -96], zoom_start=4, tiles="cartodbpositron")

folium.Choropleth(
    geo_data=county_geojson,
    data=latest,
    columns=["countycode", "all"],
    key_on="feature.id",
    fill_color="RdPu",
    fill_opacity=0.8,
    line_opacity=0.2,
    bins=bins,
    nan_fill_color="lightgray",
    legend_name="High-Speed Internet Connections (000s)",
    reset=True
).add_to(m)

m.save("internet_usage_county_map.html")
m


In [ ]:
from google.colab import files
files.download("internet_usage_county_map.html")